# Desafio Técnico PicPay

# 1. Upload da base de dados
Essa primeira etapa consiste de fazer o upload da base de dados `airports-database.csv` para o databrics.  
  
Na página inicial do Databrics, podemos clicar em `Create Table`.  
![criando tabela](assets/1_create_table.png)  
  
A próxima etapa foi realizar o upload do arquivo no Databricks.  
![fazendo o upload no databrics](assets/2_upload.png)
  
O databrics permite que criemos uma tabela por meio da interface gráfica e por meio do notebook (opção escolhida), que será realizada logo abaixo.

Temos o nosso conjunto de dados no databrics em formato CSV, esse é o nosso dado raw (ou dado bruto), sem qualquer tipo de transformação.  
Essa tabela é composta pelas seguintes colunas:
| Coluna | Descrição |
|---|---|
| id | Um identificador único para cada registro de voo. |
| year | O ano em que o voo ocorreu (2013 neste conjunto de dados). |
| month | O mês em que o voo ocorreu (1 a 12). |
| day| O dia do mês em que o voo ocorreu (1 a 31). |
| dep_time | O horário local real de partida do voo, no formato 24 horas (hhmm). |
| sched_dep_time | O horário local programado de partida do voo, no formato 24 horas (hhmm). |
| dep_delay | A diferença entre os horários real e programado de partida do voo, em minutos. Um valor positivo indica uma partida atrasada, enquanto um valor negativo indica uma partida adiantada. |
| arr_time | O horário local real de chegada do voo, no formato 24 horas (hhmm). |
| sched_arr_time | O horário local programado de chegada do voo, no formato 24 horas (hhmm).|
| arr_delay | A diferença entre os horários real e programado de chegada do voo, em minutos. Um valor positivo indica uma chegada atrasada, enquanto um valor negativo indica uma chegada adiantada. |
| carrier | O código de duas letras da companhia aérea do voo. |
| flight | O número do voo. |
| tailnum | O identificador único da aeronave usada no voo. |
| origin | O código de três letras do aeroporto de origem do voo. |
| dest | O código de três letras do aeroporto de destino do voo. |
| air_time | A duração do voo, em minutos. |
| distance | A distância entre os aeroportos de origem e destino, em milhas. |
| hour | O componente da hora do horário programado de partida, no horário local. |
| minute | O componente dos minutos do horário programado de partida, no horário local. |
| time_hour | O horário programado de partida do voo, no formato local e de data-hora (yyyy-mm-dd hh)
| name | O nome da companhia aérea do voo. |

# 2. Carga
Aqui faremos o carregamento do arquivo para uma tabela delta na camada bronze.  
A arquitetura usada aqui é a [arquitetura medallion](https://www.databricks.com/glossary/medallion-architecture).  
![medallion architecture](assets/4_medallion_architecture.png)

In [ ]:
# File location and type
file_location = "/FileStore/tables/airports_database.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.limit(10).display()

id,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,name
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,United Air Lines Inc.
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,United Air Lines Inc.
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,American Airlines Inc.
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,JetBlue Airways
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,Delta Air Lines Inc.
5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01 05:00:00,United Air Lines Inc.
6,2013,1,1,555.0,600,-5.0,913.0,854,19.0,B6,507,N516JB,EWR,FLL,158.0,1065,6,0,2013-01-01 06:00:00,JetBlue Airways
7,2013,1,1,557.0,600,-3.0,709.0,723,-14.0,EV,5708,N829AS,LGA,IAD,53.0,229,6,0,2013-01-01 06:00:00,ExpressJet Airlines Inc.
8,2013,1,1,557.0,600,-3.0,838.0,846,-8.0,B6,79,N593JB,JFK,MCO,140.0,944,6,0,2013-01-01 06:00:00,JetBlue Airways
9,2013,1,1,558.0,600,-2.0,753.0,745,8.0,AA,301,N3ALAA,LGA,ORD,138.0,733,6,0,2013-01-01 06:00:00,American Airlines Inc.


Aqui estarei salvando como uma tabela delta na camada bronze.

In [ ]:
df.write \
  .format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .saveAsTable("bronze_airports_database")

Verificando se a nossa tabela foi corretamente criada por meio de uma consulta SQL que seleciona todos os dados da tabela e retorna as 10 primeiras linhas

In [ ]:
%sql
SELECT * FROM `bronze_airports_database` LIMIT 10;

id,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,name
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,United Air Lines Inc.
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,United Air Lines Inc.
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,American Airlines Inc.
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,JetBlue Airways
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,Delta Air Lines Inc.
5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01 05:00:00,United Air Lines Inc.
6,2013,1,1,555.0,600,-5.0,913.0,854,19.0,B6,507,N516JB,EWR,FLL,158.0,1065,6,0,2013-01-01 06:00:00,JetBlue Airways
7,2013,1,1,557.0,600,-3.0,709.0,723,-14.0,EV,5708,N829AS,LGA,IAD,53.0,229,6,0,2013-01-01 06:00:00,ExpressJet Airlines Inc.
8,2013,1,1,557.0,600,-3.0,838.0,846,-8.0,B6,79,N593JB,JFK,MCO,140.0,944,6,0,2013-01-01 06:00:00,JetBlue Airways
9,2013,1,1,558.0,600,-2.0,753.0,745,8.0,AA,301,N3ALAA,LGA,ORD,138.0,733,6,0,2013-01-01 06:00:00,American Airlines Inc.


Consultando a quantidade de registros na nossa tabela bronze

In [ ]:
%sql
SELECT COUNT(*) FROM `bronze_airports_database`;

count(1)
336776


# 3. Perguntas

**1.** Qual é o número total de voos no conjunto de dados?  
**Resposta:** O conjunto de dados possui o registro de 336.776 voos.

In [ ]:
df.count()

336776

**2.** Quantos voos foram cancelados? (Considerando que voos cancelados têm `dep_time` e `arr_time` nulos)  
**Resposta:** 8255 voos foram cancelados.

In [ ]:
from pyspark.sql.functions import col
df.filter(col("dep_time").isNull() & col("arr_time").isNull()).count()


8255

**3.** Qual é o atraso médio na partida dos voos (`dep_delay`)?  
**Resposta:** O atraso médio é de 12.64 minutos.

In [ ]:
from pyspark.sql.functions import mean
df.select(mean(col("dep_delay"))).show()

+------------------+
|    avg(dep_delay)|
+------------------+
|12.639070257304708|
+------------------+



4. Quais são os 5 aeroportos com maior número de pousos?

In [ ]:
df.groupBy('dest').count().sort('count', ascending=False).limit(5).show()

+----+-----+
|dest|count|
+----+-----+
| ORD|17283|
| ATL|17215|
| LAX|16174|
| BOS|15508|
| MCO|14082|
+----+-----+



**5.** Qual é a rota mais frequente (par origin-dest)?  
**Resposta:** A rota mais frequente é a de JFK para LAX com 11.262 voos.

In [ ]:
df.groupBy(['origin', 'dest']).count().sort('count', ascending=False).limit(1).show()

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   JFK| LAX|11262|
+------+----+-----+



6. Quais são as 5 companhias aéreas com maior tempo médio de atraso na chegada? (Exiba também o tempo)

In [ ]:
from pyspark.sql import functions as F
df.groupBy('name').agg(F.mean('arr_delay')).sort('avg(arr_delay)', ascending=False).limit(5).show()

+--------------------+------------------+
|                name|    avg(arr_delay)|
+--------------------+------------------+
|Frontier Airlines...|21.920704845814978|
|AirTran Airways C...|20.115905511811025|
|ExpressJet Airlin...| 15.79643108710965|
|  Mesa Airlines Inc.|15.556985294117647|
|SkyWest Airlines ...|11.931034482758621|
+--------------------+------------------+



**7.** Qual é o dia da semana com maior número de voos?  
**Resposta:** O dia da semana com mais voos é Segunda.

In [ ]:
# criando nova coluna
def get_weekday(date):
    """
    Retorna o dia da semana, onde Segunda é 0 e Domingo é 6
    """
    date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    return date.weekday()

udf_function = udf(lambda date: get_weekday(date))

new_df = df.withColumn('weekday', udf_function(df["time_hour"]))

# agrupando por dia da semana, contando a quantidade e retornando o resultado com o maior valor
new_df.groupBy('weekday').count().sort('count', ascending=False).limit(1).show()

+-------+-----+
|weekday|count|
+-------+-----+
|      0|50690|
+-------+-----+



8. Qual o percentual mensal dos voos tiveram atraso na partida superior a 30 minutos?

In [ ]:
from pyspark.sql.functions import col, count, when, round

monthly_counts = df.groupBy("month") \
    .agg(
        count("*").alias("total_count"),
        count(when(col("dep_delay") > 30, True)).alias("greater_than_30_count")
    )

# Step 2: Calculate the percentage
monthly_percentage = monthly_counts.withColumn(
    "percentage", round((col("greater_than_30_count") / col("total_count")) * 100, 2)
)

# Show the result
monthly_percentage.sort("percentage", ascending=False).show()

+-----+-----------+---------------------+----------+
|month|total_count|greater_than_30_count|percentage|
+-----+-----------+---------------------+----------+
|    7|      29425|                 6173|     20.98|
|    6|      28243|                 5717|     20.24|
|   12|      28135|                 4871|     17.31|
|    4|      28330|                 4531|     15.99|
|    5|      28796|                 4416|     15.34|
|    3|      28834|                 4309|     14.94|
|    8|      29327|                 4238|     14.45|
|    2|      24951|                 3182|     12.75|
|    1|      27004|                 3350|     12.41|
|   10|      28889|                 2697|      9.34|
|    9|      27574|                 2419|      8.77|
|   11|      27268|                 2388|      8.76|
+-----+-----------+---------------------+----------+



**9.** Qual a origem mais comum para voos que pousaram em Seattle (SEA)?  
**Resposta:** A origem mais comum é de JFK.

In [ ]:
sea_dest = df.filter(df.dest == "SEA")
sea_dest.groupBy("origin").count().sort("count", ascending=False).limit(1).show()

+------+-----+
|origin|count|
+------+-----+
|   JFK| 2092|
+------+-----+



**10.** Qual é a média de atraso na partida dos voos (`dep_delay`) para cada dia da semana?

In [ ]:
from pyspark.sql import functions as F
new_df.groupBy('weekday').agg(F.mean('dep_delay')).sort('avg(dep_delay)', ascending=False).show()

+-------+------------------+
|weekday|    avg(dep_delay)|
+-------+------------------+
|      3|16.148919990957108|
|      0|14.778936729330908|
|      4| 14.69605749486653|
|      2|11.803512219083876|
|      6|11.589531801152422|
|      1|10.631682565455652|
|      5| 7.650502333676133|
+-------+------------------+



**11.** Qual é a rota que teve o maior tempo de voo médio (`air_time`)?  
**Resposta:** A rota com o maior tempo é a rota JFK -> HNL com tempo médio de 623 minutos.

In [ ]:
new_df.groupBy(["origin", "dest"]).agg(F.mean('air_time')).sort('avg(air_time)', ascending=False).limit(1).show()

+------+----+-----------------+
|origin|dest|    avg(air_time)|
+------+----+-----------------+
|   JFK| HNL|623.0877192982456|
+------+----+-----------------+



12. Para cada aeroporto de origem, qual é o aeroporto de destino mais comum?

In [ ]:
from pyspark.sql.functions import col, count, row_number
from pyspark.sql.window import Window

# Step 1: Count occurrences of each destination for each origin
destination_counts = df.groupBy("origin", "dest").agg(
    count("*").alias("count")
)

# Step 2: Find the most common destination for each origin
window_spec = Window.partitionBy("origin").orderBy(col("count").desc())
most_common_destination = destination_counts.withColumn(
    "rank", row_number().over(window_spec)
).filter(col("rank") == 1).drop("rank")

# Show the result
most_common_destination.show()

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   EWR| ORD| 6100|
|   JFK| LAX|11262|
|   LGA| ATL|10263|
+------+----+-----+



13. Quais são as 3 rotas que tiveram a maior variação no tempo médio de voo (`air_time`)?

14. Qual é a média de atraso na chegada para voos que tiveram atraso na partida superior a 1 hora?

15. Qual é a média de voos diários para cada mês do ano?

16. Quais são as 3 rotas mais comuns que tiveram atrasos na chegada superiores a 30 minutos?

17. Para cada origem, qual o principal destino?

# 4. Enriquecimento da base de dados
Duas APIs externas serão usadas para enriquecer a nossa base de dados
* [Weatherbit API](https://www.weatherbit.io/): Fornece dados históricos sobre as condições meteorológicas.
* [AirportDB API](https://airportdb.io/): Fornece informações detalhadas sobre aeroportos, incluindo
coordenadas geográficas.

Primeiramente, iremos coletar os dados dos aeroportos, para isso irei retornar todos os valores únicos das colunas `origin` e `dest`.

In [ ]:
airports = df.select("origin").distinct().union(df.select("dest").distinct()).distinct()
airports.show(10)

+------+
|origin|
+------+
|   LGA|
|   EWR|
|   JFK|
|   PSE|
|   MSY|
|   SNA|
|   BUR|
|   GRR|
|   MYR|
|   GSO|
+------+
only showing top 10 rows



In [ ]:
import requests

def request_airportDB(airport):
    airportdb_key = 'YOUR_API_KEY'

    response = requests.get(url=f'https://airportdb.io/api/v1/airport/K{airport}?apiToken={airportdb_key}')
    return response.json()

air = request_airportDB("JFK")

In [ ]:
air

{'ident': 'KJFK',
 'type': 'large_airport',
 'name': 'John F Kennedy International Airport',
 'latitude_deg': 40.639801,
 'longitude_deg': -73.7789,
 'elevation_ft': '13',
 'continent': 'NA',
 'iso_country': 'US',
 'iso_region': 'US-NY',
 'municipality': 'New York',
 'scheduled_service': 'yes',
 'gps_code': 'KJFK',
 'iata_code': 'JFK',
 'local_code': 'JFK',
 'home_link': 'https://www.jfkairport.com/',
 'wikipedia_link': 'https://en.wikipedia.org/wiki/John_F._Kennedy_International_Airport',
 'keywords': 'Manhattan, New York City, NYC, Idlewild',
 'icao_code': 'KJFK',
 'runways': [{'id': '244968',
   'airport_ref': '3622',
   'airport_ident': 'KJFK',
   'length_ft': '12079',
   'width_ft': '200',
   'surface': 'Concrete - Grooved',
   'lighted': '1',
   'closed': '0',
   'le_ident': '04L',
   'le_latitude_deg': '40.622',
   'le_longitude_deg': '-73.7856',
   'le_elevation_ft': '12',
   'le_heading_degT': '31',
   'le_displaced_threshold_ft': '',
   'he_ident': '22R',
   'he_latitude_deg'

In [ ]:
def request_weatherbit(lat, lon, start_date, end_date):
    weatherbit_key = "YOUR_API_KEY"

    url = 'https://api.weatherbit.io/v2.0/history/daily'

    params = {
        'lat': lat,
        'lon': lon,
        'start_date': start_date,
        'end_date': end_date,
        'key': weatherbit_key
    }

    headers = {
        'Accept': 'application/json'
    }

    response = requests.get(url, params=params, headers=headers)

    return response.json()

lat = 40.7128
lon = -74.0060
start_date = '2023-01-01'
end_date = '2023-01-02'

wea = request_weatherbit(lat, lon, start_date, end_date)
wea

{'city_id': '5128581',
 'city_name': 'New York City',
 'country_code': 'US',
 'data': [{'clouds': 37,
   'datetime': '2023-01-01',
   'dewpt': 5.7,
   'dhi': 27,
   'dni': 230,
   'ghi': 99,
   'max_dhi': 92,
   'max_dni': 773,
   'max_ghi': 427,
   'max_temp': 11.9,
   'max_temp_ts': 1672603200,
   'max_uv': 2.2,
   'max_wind_dir': 120,
   'max_wind_spd': 5,
   'max_wind_spd_ts': 1672588800,
   'min_temp': 7.3,
   'min_temp_ts': 1672624800,
   'precip': 0.4,
   'precip_gpm': 0.4,
   'pres': 1010,
   'revision_status': 'final',
   'rh': 77,
   'slp': 1013,
   'snow': 0,
   'snow_depth': None,
   'solar_rad': 97,
   't_dhi': 639,
   't_dni': 5510,
   't_ghi': 2378,
   't_solar_rad': 2317,
   'temp': 9.8,
   'ts': 1672549200,
   'wind_dir': 120,
   'wind_gust_spd': 8.2,
   'wind_spd': 3}],
 'lat': 40.7128,
 'lon': -74.006,
 'sources': ['997271-99999',
  '720553-99999',
  'imerg',
  'era5',
  'modis',
  'snodas'],
 'state_code': 'NY',
 'station_id': '720553-99999',
 'timezone': 'America/N